In [1]:
from hyptorch.geoopt import PoincareBall   
import torch


manifold = PoincareBall(c=1.0, learnable=True)
vis_embed = torch.rand(100, 12,256)
text_embed = torch.rand(100, 1, 256)
embed = manifold.expmap0(vis_embed)
manifold.assert_check_point_on_manifold(embed)

# model = FuseEncoder(
#     config,           
#     d_visions=[100, 100], 
#     d_texts=[100, 100], 
#     ft_out=256, 
#     vision_bodies=[], 
#     text_bodies=[],
#     vision_head=None,
#     text_head=None,
#     mapper=None,
#     manifold=None, 
# )

torch.Size([40, 12, 32])
torch.Size([40, 40, 12, 32])


In [35]:
sim_t2q = torch.matmul(
    a, b.permute(0, 2, 1)
).squeeze()

In [36]:
sim_t2q.max(-1).values.shape

torch.Size([40, 40])